In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pandas as pd
import os

MODEL_PATH = "../checkpoints/distilbert_sentiment"
RESULTS_DIR = "../results"
OUTPUT_FILE = os.path.join(RESULTS_DIR, "demo_predictions.csv")

if not os.path.exists(MODEL_PATH):
    print(f"Error: Model not found at {MODEL_PATH}")
    print("Please download the model weights and place them in the checkpoints folder.")
else:
    print(f"Loading model from: {MODEL_PATH}")

try:
    tokenizer = DistilBertTokenizer.from_pretrained(MODEL_PATH)
    model = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval() 
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load model: {e}")

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
    
    pred_label = torch.argmax(logits, dim=1).item()
    confidence = probabilities[0][pred_label].item()
    
    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
    
    return label_map[pred_label], confidence

sample_inputs = [
    "I absolutely love this new phone! The camera is amazing.",  # Positive
    "The food was cold and the service was terrible.",           # Negative
    "I am going to work by bus today.",                          # Neutral
    "Can't wait for the weekend! So excited.",                   # Positive
    "This is the worst experience I've ever had.",               # Negative
    "The weather is okay, not too bad.",                         # Neutral/Positive
]

results = []

print("\n--- 🚀 Running Demo Predictions ---")
for text in sample_inputs:
    sentiment, conf = predict_sentiment(text)
    print(f"Text: '{text}'")
    print(f"Prediction: {sentiment} (Confidence: {conf:.4f})\n")
    
    results.append({
        "input_text": text,
        "predicted_sentiment": sentiment,
        "confidence": conf
    })

if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

df = pd.DataFrame(results)
df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Results saved to: {OUTPUT_FILE}")

df

Loading model from: ../checkpoints/distilbert_sentiment
✅ Model loaded successfully!

--- 🚀 Running Demo Predictions ---
Text: 'I absolutely love this new phone! The camera is amazing.'
Prediction: Positive (Confidence: 0.9901)

Text: 'The food was cold and the service was terrible.'
Prediction: Negative (Confidence: 0.9058)

Text: 'I am going to work by bus today.'
Prediction: Neutral (Confidence: 0.7785)

Text: 'Can't wait for the weekend! So excited.'
Prediction: Positive (Confidence: 0.9872)

Text: 'This is the worst experience I've ever had.'
Prediction: Negative (Confidence: 0.9329)

Text: 'The weather is okay, not too bad.'
Prediction: Positive (Confidence: 0.7129)

✅ Results saved to: ../results\demo_predictions.csv


,input_text,predicted_sentiment,confidence
0,I absolutely love this new phone! The camera i...,Positive,0.990069
1,The food was cold and the service was terrible.,Negative,0.905779
2,I am going to work by bus today.,Neutral,0.778529
3,Can't wait for the weekend! So excited.,Positive,0.987156
4,This is the worst experience I've ever had.,Negative,0.932872
5,"The weather is okay, not too bad.",Positive,0.712889
